# 《神经网络与深度学习》学习笔记
## 第4章 前馈神经网络

### 4.1 神经元
* 净输入：$ z = \mathbf{w}^{\mathrm{T}}\mathbf{x} + b$
* 激活函数：$f(*)$
  * 连续可导(允许少数点上不可导)的非线性函数。
  * 尽可能简单以提高计算效率。
  * 激活函数的导函数的值域需要在一个合适的区间内。
* 活性值：$a = f\left(z\right)$

#### 4.1.1 Sigmoid型激活函数
* Sigmoid型激活函数指的是一类S型曲线函数。
* Logistic函数：$\sigma(x)=\frac{1}{1+\exp (-x)}$
* Tanh函数：$\mathrm{tanh}\left(x\right)=\frac{\exp\left(x\right)-\exp\left(-x\right)}{\exp\left(x\right)+\exp\left(-x\right)}$
* $\mathrm{tanh}\left(x\right)=2\sigma(2x)-1$
* Hard-Logistic和Hard-Tanh函数：在一定区域内用一阶泰勒占来代替。

#### 4.1.2 修正单元
* $\mathrm{ReLU}(x) = \max(0, x)$
* $\mathrm{LeakyReLU}(x) = \max(x, \gamma x)$
* $\mathrm{PReLU}(x) = \max(0, x) + \gamma_{i}\min(0,x)$
* $\mathrm{ELU}(x) = \max(0, x) + \min\left(0, \gamma\left(\exp\left(x\right)-1\right)\right)$
* $\mathrm{Softplus}(x) = \log(1 + \exp(x))$

#### 4.1.3 Swish函数
* $\mathrm{swish}(x)=x\sigma(\beta x)$
* $\beta$为可学习的超参数，当$\sigma(\beta x)$接近1时，激活函数值接近$x$，当$\sigma(\beta x)$接近0时，激活函数的输出近似于0。

#### 4.1.4 高斯误差线性单元
* 利用正态分布的累计分布函数。

#### 4.1.5 Maxout单元
* $\mathrm{maxout}(x) = \max_{k\in[1,K]}(z_{k})$

### 4.2 网络结构
* 前馈网络、记忆网络、图网络

### 4.3 前馈神经网络
* 组成：输入层、隐藏层、输出层
* $\mathbf{z}^{(l)} = W^{(l)}\cdot\mathbf{a}^{(l-1)}+\mathbf{b}^{(l)}$
* $\mathbf{a}^{(l)}=f_l(\mathbf{z}^{(l)})$

#### 4.3.1 通用近似定理
* 常见的连续非线性函数都可以用前馈神经网络来近似。

#### 4.3.2 应用到机器学习
* 最后一层设置$C$个神经元，其激活函数为softmax函数，输出的值可以作为每个类的后验概率。

#### 4.3.3 参数学习
* 使用交叉熵损失函数，来迭代更新参数。
* 对每个参数进行求偏导比较低效，一般使用反向传播算法。

### 4.4 反向传播算法
* 前馈计算每一层的净输入$\mathbf{z}^{(l)}$和激活值$\mathbf{a}^{(l)}$，直到最后一层；
* 反向传播计算每一层的误差项$\delta^{(l)}$；
* 计算每一层参数的偏导数，并更新参数。

### 4.5 自动梯度计算
* 数值微分：利用数值方法来计算函数的导数。
* 符号微分：基于符号计算的自动求导方法。
* 自动微分：符号微分的程序。

### 4.6 优化问题
* 非凸优化问题
* 梯度小时问题

In [5]:
# 前馈神经网络Pytorch实现
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

batch_size = 64
learning_rate = 1e-2
num_epochs = 50
use_gpu = torch.cuda.is_available()

# 下载训练集 MNIST 手写数字训练集
train_dataset = datasets.FashionMNIST(
    root='./datasets', train=True,
    transform=transforms.ToTensor(), download=True)

test_dataset = datasets.FashionMNIST(
    root='./datasets', train=False,
    transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [8]:
# 定义简单的前馈神经网络
class NeuralNetwork(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(in_dim, n_hidden_1),
            nn.ReLU(True))
        self.layer2 = nn.Sequential(
            nn.Linear(n_hidden_1, n_hidden_2),
            nn.ReLU(True))
        self.layer3 = nn.Sequential(
            nn.Linear(n_hidden_2, out_dim),
            nn.ReLU(True))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x


In [14]:
model = NeuralNetwork(28 * 28, 100, 30, 10)
if use_gpu:
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
for epoch in range(num_epochs):
    print('*' * 10)
    print(f'epoch {epoch+1}')
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.view(img.size(0), -1)
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        # 向前传播
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.item()
        _, pred = torch.max(out, 1)
        running_acc += (pred == label).float().mean()
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print(f'[{epoch+1}/{num_epochs}] Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    print(f'Finish {epoch+1} epoch, Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for data in test_loader:
        img, label = data
        img = img.view(img.size(0), -1)
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        with torch.no_grad():
            out = model(img)
            loss = criterion(out, label)
        eval_loss += loss.item()
        _, pred = torch.max(out, 1)
        eval_acc += (pred == label).float().mean()
    print(f'Test Loss: {eval_loss/len(test_loader):.6f}, Acc: {eval_acc/len(test_loader):.6f}\n')

# 保存模型
torch.save(model.state_dict(), './neural_network.pth')

**********
epoch 1
[1/50] Loss: 2.271728, Acc: 0.187240
[1/50] Loss: 2.182293, Acc: 0.286380
[1/50] Loss: 2.045416, Acc: 0.342483
Finish 1 epoch, Loss: 2.025991, Acc: 0.348664
Test Loss: 1.566071, Acc: 0.479996

**********
epoch 2
[2/50] Loss: 1.468649, Acc: 0.491719
[2/50] Loss: 1.408114, Acc: 0.500130
[2/50] Loss: 1.367240, Acc: 0.512483
Finish 2 epoch, Loss: 1.360993, Acc: 0.515192
Test Loss: 1.266726, Acc: 0.537619

**********
epoch 3
[3/50] Loss: 1.236760, Acc: 0.559271
[3/50] Loss: 1.221126, Acc: 0.567005
[3/50] Loss: 1.200834, Acc: 0.575660
Finish 3 epoch, Loss: 1.197690, Acc: 0.576892
Test Loss: 1.167985, Acc: 0.591262

**********
epoch 4
[4/50] Loss: 1.139646, Acc: 0.597240
[4/50] Loss: 1.119782, Acc: 0.602266
[4/50] Loss: 1.113937, Acc: 0.603229
Finish 4 epoch, Loss: 1.112869, Acc: 0.603611
Test Loss: 1.100582, Acc: 0.600219

**********
epoch 5
[5/50] Loss: 1.078928, Acc: 0.612344
[5/50] Loss: 1.075522, Acc: 0.612839
[5/50] Loss: 1.071083, Acc: 0.613663
Finish 5 epoch, Loss: 

KeyboardInterrupt: 